In [1]:
# sigmoid 계층 
# sigmoid 계층의 역전파는 순전파의 출력(y)만으로 계산 가능

class Sigmoid: 
    def __init__(self):
        self.out = None

    def forward(self, x): # 순전파의 출력을 인스턴스 변수 out에 보관
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout): # 역전파 때 순전파 출력 out에 보관한거 사용
        dx = dout * (1.0 - self.out) * self.out

        return dx

신경망의 순전파 → 가중치 신호의 총합 계산 때문에 행렬의 곱 사용 [넘파이에선 np.dot()]

In [4]:
# 각각 형상이 다차원 배열

import numpy as np

x = np.random.rand(2)    # 입력
w = np.random.rand(2, 3) # 가중치
b = np.random.rand(3)    # 편향

print(x.shape)
print(w.shape)
print(b.shape)

y = np.dot(x, w) + b # 뉴런의 가중치 합

# 신경망 순전파의 흐름 → y를 활성화 함수로 변환하여 다음 층으로 전파하는 것

(2,)
(2, 3)
(3,)


#### • 행렬 곱의 계산
: 대응하는 차원의 원소 수를 일치시키는게 핵심

• x와 w의 곱은 대응하는 차원의 원소 수를 일치시켜야 한다. 

• 행렬의 형상이 (2, 3)처럼 표시하는 이유 → 넘파이의 shape 함수 출력과 형태 통일하기 위해

In [8]:
# 배치용 Affine 계층

x_dot_w = np.array([[0, 0, 0],[10, 10, 10]])
b = np.array([1, 2, 3])

print(x_dot_w)

print(x_dot_w + b)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [10]:
# 역전파는 각 데이터의 역전파 값이 편향의 원소에 모여야 한다. 

dy = np.array([[1, 2, 3], [4, 5, 6]])
print(dy)

db = np.sum(dy, axis=0) # 0번째 축에 대해 axis=0의 총합을 구하는 것
print(db)

# 데이터가 2개(n = 2)
# 편향의 역전파는 두 데이터에 대한 미분을 데이터마다 더해서 구한다. 

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [ ]:
# Affine 구현 

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None 
        
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

#### 소프트맥스 
: 입력 값을 정규화하여 출력

####
∙ 입력 값을 정규화(출력의 합이 1이 되게)하여 출력

#### 신경망 학습의 목적
: 신경망의 출력(softmax)이 정답 레이블과 가까워지도록 가중치 매개변수의 값 조정하는 것

: 신경망의 출력과 정답 레이블 오차를 효율적으로 앞 계층에 전달

In [ ]:
# softmax_with_Loss

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

# 역전파 때는 전파하는 값을 배치의 수(batch_size)로 나눠서 데이터 1개당 오차를 앞 계층으로 전파

### 신경망 학습 전체 그림

- 전제
: 신경망엔 적응 가능한 가중치, 편향이 있다. 총 4단계로 수행

1. 미니배치 
: 훈련 데이터 중 일부를 무작위로 가져옴 / 미니배치의 손실 함수 값 줄이는 것이 목표
###
2. 기울기 산출
: 미니배치의 손실 함수 값 줄이기 위해 각 가중치 매개변수의 기울기 구하기.
: 기울기는 손실 함수의 값을 가장 작게 하는 방향 제시
###
3. 매개변수 갱신 
: 가중치 매개변수 기울기 방향으로 아주 조금 갱신
###
4. 반복 
: 1~3단계 반복

In [13]:
# 앞 장과 다른 부분은 계층을 사용한다. 
# 인식 결과 얻는 처리 (predict()), 기울기 구하는 처리(gradient())

import sys, os
sys.path.append('/Users/krc/Downloads/deep-learning-from-scratch-master')
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict
 
 
class TwoLayerNet:
 
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)
 
        # 계층 생성
        # OrderedDict = 순서가 있는 딕셔너리, 순서 기억
        # 순전파 때는 계층을 추가한 순서대로 / 역전파 때는 계층 반대 순서로 호출
        self.layers = OrderedDict() # 순서가 있는 딕셔너리 
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
 
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x: 입력데이터, t : 정답레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x: 입력데이터, t : 정답레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward, 순전파
        self.loss(x, t)
 
        # backward, 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
 
        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
 
        return grads


### 기울기 구하는 방법
1. 수치 미분을 써서 구하는 방법

2. 해석적으로 수식을 풀어 구하는 방법  → 오차역전파법 이용하여 매개변수가 많아도 효율적으로 계산 

###
• 기울기 확인 
: 두 방식으로 구한 기울기가 일치함 (거의 같음)

In [17]:
import sys, os
sys.path.append('/Users/krc/Downloads/deep-learning-from-scratch-master/ch05')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

# 수치 미분으로 구한 기울기와 오차역전파법으로 구한 기울기의 오차 확인
# 각 가중치 매개변수 차이의 절댓값 구하고 평균한 값이 오차가 된다. 
# → 수치 미분과 오차역전파법으로 구한 기울기의 차이가 매우 작다

W1:1.8440637058490597e-06
b1:1.5782359941071237e-05
W2:4.784549472707054e-09
b2:1.4006088839835228e-07


In [18]:
# 오차역전파법을 사용한 신경망 학습 
# 기울기를 오차역전파법으로 구한다는 점

import sys, os
sys.path.append('/Users/krc/Downloads/deep-learning-from-scratch-master/ch05')

import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.13926666666666668 0.1425
0.90425 0.9096
0.9218833333333334 0.9248
0.9317833333333333 0.9314
0.9435166666666667 0.9424
0.9502166666666667 0.9499
0.955 0.9523
0.9572166666666667 0.9544
0.9639333333333333 0.9578
0.9659 0.96
0.9689833333333333 0.963
0.9713833333333334 0.9637
0.9743833333333334 0.9647
0.97505 0.9671
0.9748833333333333 0.9658
0.9766666666666667 0.9675
0.9785333333333334 0.9681


## 5장 정리

#### 계산 그래프를 이용하여 신경망의 동작과 오차역전파법 설명 → 처리 과정을 계층이라는 단위로 구현

ex) ReLU 계층, Softmax-with-Loss 계층, Affine 계층, Softmax 계층 등 
###
- 모든 계층에서 forward, backward 메서드 구현
###
- 전자는 데이터를 순방향 전파, 후자는 역방향으로 전파  → 가중치 매개변수의 기울기 효율적으로 구함

### ▶︎ 이처럼 동작을 계층으로 모듈화한 덕분에, 신경망 계층을 자유롭게 조합하여 원하는 신경망 쉽게 만들 수 있음

### • 이번 장에서 배운 내용

1. 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다.

2. 계산 그래프 노드는 국소적 계산으로 구성

3. 순전파는 통상 계산 수행 / 역전파로는 각 노드의 미분 구할 수 있다. 

4. 신경망의 구성 요소를 계층으로 구현하여 기울기 효율적으로 계산 (오차역전파법)

5. 수치 미분과 오차역전파법 결과 비교하연 오차역전파법의 구현에 잘못 없는지 확인 가능